# Schema Discovery

Collecting schema information from the database definitions.
Intended to extract the schema existing Reporting metadata for further examination.

In [1]:
// install packages here:
#r "nuget: Microsoft.Data.SqlClient, 5.1.4"
#r "nuget: Newtonsoft.Json, 13.0.3"

Installed Packages Microsoft.Data.SqlClient, 5.1.4 Newtonsoft.Json, 13.0.3

In [2]:
// General usings (needed for intellisense to work below)
using Microsoft.Data.SqlClient;
using Newtonsoft.Json;
using System.IO;

### Verify the connections work

Adjust the credentials if necessary.

In [3]:
// verify the connection credentials work
internal string credentials  = System.Environment.GetEnvironmentVariable("NIX_DB_DG");
try
{
    using (var connection = new Microsoft.Data.SqlClient.SqlConnection(credentials))
    {   connection.Open();
        var x = connection.State;
        Console.WriteLine($"Connection: {x} succeeded."); 
    }
}
catch (Exception ex)
{
    Console.WriteLine($"{ex.Message}"); 
}

Connection: Open succeeded.


### Support code

Functions and classes to provide lookup functionality

In [4]:
// A function to prettify json
internal string Prettify(string json)
{
   return Newtonsoft.Json.Linq.JToken.Parse(json).ToString(Newtonsoft.Json.Formatting.Indented);
}
// Let's define some classes to manage our schema information

public class TableDefinition
{
    public string SchemaName { get; set; }
    public string TableName {get; set;}
    public List<ColumnDefinition> Columns {get; set; }
}
public class ColumnDefinition
{
    public string ColumnName {get; set; }
    public string DataType { get; set; }
    public bool IsNullable {get; set; }
    public int Size { get; set; }
}
// A function to retrieve schema information for one table
internal string GetTableSchema(string schemaName, string tableName)
{
    string sql = 
        $"select '{schemaName}' SchemaName, '{tableName}' TableName, "
        +"(select column_name ColumnName, data_type DataType, "
        +"case when is_nullable='yes' then 1 else 0 end IsNullable, "
        +"isnull(character_maximum_length, 0) Size "
        +$"from information_schema.columns where TABLE_SCHEMA = '{schemaName}' and "
        +$"TABLE_NAME = '{tableName}' order by ordinal_position "
        +"for json path) Columns for json path, "
        +"WITHOUT_ARRAY_WRAPPER"; // because we are only getting a single result

    // Console.WriteLine(sql);
    using (var connection = new SqlConnection(credentials))
    {
        using (var command = new SqlCommand(sql, connection))
        {
            connection.Open();
            var result = new StringBuilder();
            var reader = command.ExecuteReader();
            if (!reader.HasRows)
            {
                result.Append("{}");// empty object
            }
            else
            {
                while (reader.Read())
                {
                    result.Append(reader.GetValue(0).ToString());
                }
            }
            return result.ToString();
        }
    }
}
    
public TableDefinition GetTableDefinition(string schemaName, string tableName)
{
   // Test against a single table
   var x = GetTableSchema(schemaName, tableName);
   // Console.WriteLine(Prettify(x));
   return Newtonsoft.Json.JsonConvert.DeserializeObject<TableDefinition>(x);
}

### Test against a couple of tables


In [5]:
List<TableDefinition> tableDefinitions = new List<TableDefinition>();
tableDefinitions.Add(GetTableDefinition("rpt", "Reports"));
tableDefinitions.Add(GetTableDefinition("rpt", "ReportDataTypes"));
tableDefinitions.Add(GetTableDefinition("rpt", "ReportColumns"));

tableDefinitions.Display();

index value 0 Submission#5+TableDefinition SchemaName rpt TableName Reports Columns index value 0 Submission#5+ColumnDefinition ColumnName ReportId DataType bigint IsNullable False Size 0 1 Submission#5+ColumnDefinition ColumnName OrganizationId DataType bigint IsNullable False Size 0 2 Submission#5+ColumnDefinition ColumnName ReportType DataType varchar IsNullable False Size 100 3 Submission#5+ColumnDefinition ColumnName FileNameTemplate DataType varchar IsNullable True Size 500 4 Submission#5+ColumnDefinition ColumnName Enabled DataType bit IsNullable False Size 0 5 Submission#5+ColumnDefinition ColumnName Format DataType varchar IsNullable True Size 25 6 Submission#5+ColumnDefinition ColumnName TextQualifier DataType varchar IsNullable True Size 25 7 Submission#5+ColumnDefinition ColumnName TransmitterConfiguration DataType nvarchar IsNullable True Size -1 8 Submission#5+ColumnDefinition ColumnName Parameters DataType nvarchar IsNullable True Size 2000 9 Submission#5+ColumnDefinition ColumnName ReportMethodId DataType int IsNullable True Size 0 10 Submission#5+ColumnDefinition ColumnName ReportTitle DataType nvarchar IsNullable True Size 100 11 Submission#5+ColumnDefinition ColumnName DateModified DataType datetime2 IsNullable True Size 0 12 Submission#5+ColumnDefinition ColumnName ReportInstanceId DataType bigint IsNullable True Size 0 13 Submission#5+ColumnDefinition ColumnName IsScheduledInApp DataType bit IsNullable True Size 0 1 Submission#5+TableDefinition SchemaName rpt TableName ReportDataTypes Columns index value 0 Submission#5+ColumnDefinition ColumnName ReportDataTypeId DataType bigint IsNullable False Size 0 1 Submission#5+ColumnDefinition ColumnName LocalizationKey DataType varchar IsNullable False Size 100 2 Submission#5+ColumnDefinition ColumnName CreatedDate DataType datetime2 IsNullable False Size 0 3 Submission#5+ColumnDefinition ColumnName ModifiedDate DataType datetime2 IsNullable True Size 0 2 Submission#5+TableDefinition SchemaName rpt TableName ReportColumns Columns index value 0 Submission#5+ColumnDefinition ColumnName ReportColumnId DataType bigint IsNullable False Size 0 1 Submission#5+ColumnDefinition ColumnName ReportDataTypeId DataType bigint IsNullable False Size 0 2 Submission#5+ColumnDefinition ColumnName LocalizationKey DataType varchar IsNullable False Size 100 3 Submission#5+ColumnDefinition ColumnName TableName DataType varchar IsNullable False Size 100 4 Submission#5+ColumnDefinition ColumnName ColumnName DataType varchar IsNullable False Size 100 5 Submission#5+ColumnDefinition ColumnName CreatedDate DataType datetime2 IsNullable False Size 0 6 Submission#5+ColumnDefinition ColumnName ModifiedDate DataType datetime2 IsNullable True Size 0

### Produce a document with the results

The intent is to be able to capture the metadata to facilitate analysis of our data structures and use.
We can assume that any tool we develop should be able to ingest the table definitions in the form of json data, so that is what we will produce.


In [6]:
string[] tablenames = new string[]
{
    "Content",
    "ContentDurationInMinutes",
    "DailyUserActivity",
    "DefaultDateRangeOptions",
    "LearningCampaignEventCounts",
    "MessageQueue",
    "MessageQueue_PreProcessing",
    "PathwayInsightsKPIs",
    "PopularContentTags",
    "PopularContentTags_Switch",
    "PopularContentTags_Switch",
    "PopularContentTags_Switch_Empty",
    "PopularSearchTerms",
    "PopularUserTags",
    "RelatedContent",
    "RelatedContent_Partition",
    "ReportActivityLog",
    "ReportAggregateTypes",
    "ReportCategories",
    "ReportColumns",
    "ReportDataTypeOperators",
    "ReportDataTypes",
    "ReportDefinitionColumns",
    "ReportDefinitionCustomAttributes",
    "ReportDefinitions",
    "ReportDownloads",
    "ReportExecutionLog",
    "ReportFailureTypeRegexMatches",
    "ReportFailureTypes",
    "ReportFields",
    "ReportFileLog",
    "ReportInstances",
    "ReportLogMetadata",
    "ReportLogs",
    "ReportMethods",
    "ReportOperatorTypes",
    "Reports",
    "ReportSchedule",
    "ReportSearchableTermTypes",
    "ReportTestBatch",
    "ReportTestBatchLog",
    "ReportTimeRanges",
    "ReportTransmissionLogs",
    "ReportTransmitterConfigLinks",
    "ReportTransmitterConfigs",
    "ReportTypes",
    "ReportUserSegments",
    "Salty",
    "SubReports",
    "TableSyncTracking",
    "TransmissionConfig",
    "TrendingContent",
    "TrendingContent_Partition",
    "UserCollectionsStatus",
    "UserContentView_Archive",
    "UserDailyContentViewCount",
    "UserInputCompletions",
    "UserLogins",
    "UserSearchTerms",
    "UserSummary",
    "UserViewBrowsers",
    "UserViewLocations",
    "UserViewOSs",
    "UserViewPlatforms"
};
tableDefinitions = new List<TableDefinition>();
foreach (string tableName in tablenames)
{ 
    tableDefinitions.Add(GetTableDefinition("rpt", tableName));
}


### Serialize the object to a json file

Adjust the file path to suit.

In [7]:
string path = @".\data\Rpt-Schema-Definitions.json";
internal async Task WriteOutput()
{ 
    await using FileStream output = File.Create(path);
    await System.Text.Json.JsonSerializer.SerializeAsync(output, tableDefinitions);
}
await WriteOutput(); 

### Next steps

The file can now be re-created as needed, commenting out any table names that are not necessary.

As a refinement step, we can add Global Key data to individual columns for natural joining - this is something we will need in future for auto-generating snowflake queries.

These can also be used to clarify table linkages.

